# 张量
## 张量类型：
- torch.FloatTensor 32位浮点型
- torch.DoubleTensor 64位浮点型
- torch.ShortTensor 16位整形
- torch.IntTensor 32位整形
- torch.LongTensor 64位整形
## 创建常量

### torch.Tensor()创建的是torch.FloatTensor类型

In [1]:
import torch
a  = torch.Tensor([[1, 2],[2, 3]])
a

tensor([[1., 2.],
        [2., 3.]])

In [2]:
b = torch.LongTensor([[2],[4]])
b

tensor([[2],
        [4]])

In [3]:
# 创建全零tensor
torch.zeros((2,3))

tensor([[0., 0., 0.],
        [0., 0., 0.]])

**可以像numpy一样通过索引方式获取其中的元素，也可以改变它的值**

In [4]:
a[0][0] = 100
a

tensor([[100.,   2.],
        [  2.,   3.]])

In [5]:
numpy_a = a.numpy()
numpy_a

array([[100.,   2.],
       [  2.,   3.]], dtype=float32)

**从numpy中创建tensor**

In [6]:
import numpy as np
b = np.arange(24).reshape(2, 3, 4)
print(b)
tensor_b = torch.from_numpy(b)
tensor_b

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]], dtype=torch.int32)

### tensor存放的位置CPU或者GPU

In [7]:
# 查看是否支持GPU
torch.cuda.is_available()

True

In [8]:
tensor_b.device

device(type='cpu')

In [9]:
gpu_b = tensor_b.cuda()
gpu_b.device

device(type='cuda', index=0)

# 创建变量
变量和张量本身没什么区别，只是变量会被放到计算图中，然后进行前向、反向传播，自动求导。

**变量的三个属性**
<img src='../images/001.png'>

### 当对求导的变量是标量时

In [10]:
x = torch.autograd.Variable(torch.tensor(1, dtype=torch.float), requires_grad=True)

In [11]:
w = torch.autograd.Variable(torch.tensor(2, dtype=torch.float), requires_grad=True)

In [12]:
b = torch.autograd.Variable(torch.tensor(3, dtype=torch.float), requires_grad=True)

In [13]:
y = w * x + b

In [14]:
print(x.data, x.grad, x.grad_fn)
print(w.data, w.grad, w.grad_fn)
print(b.data, b.grad, b.grad_fn)
print(y.data, y.grad, y.grad_fn)

tensor(1.) None None
tensor(2.) None None
tensor(3.) None None
tensor(5.) None <AddBackward0 object at 0x0000024A15B158C8>


In [15]:
y.backward()

**等价于y.backward(torch.FloatTensor([1]))**

In [16]:
print(x.data, x.grad, x.grad_fn)
print(w.data, w.grad, w.grad_fn)
print(b.data, b.grad, b.grad_fn)
print(y.data, y.grad, y.grad_fn)

tensor(1.) tensor(2.) None
tensor(2.) tensor(1.) None
tensor(3.) tensor(1.) None
tensor(5.) None <AddBackward0 object at 0x0000024A15B0E108>


### 当对求导的变量是向量时
需要在backward（）中传入参数，为相同维数的张量

In [17]:
x = torch.randn(3)
x

tensor([-0.0804, -2.4482, -0.1980])

In [18]:
x = torch.autograd.Variable(x, requires_grad=True)

In [19]:
y = x ** 2

In [20]:
print(x , x.data, x.grad, x.grad_fn)
print(y, y.data, y.grad, y.grad_fn)

tensor([-0.0804, -2.4482, -0.1980], requires_grad=True) tensor([-0.0804, -2.4482, -0.1980]) None None
tensor([0.0065, 5.9937, 0.0392], grad_fn=<PowBackward0>) tensor([0.0065, 5.9937, 0.0392]) None <PowBackward0 object at 0x0000024A15AFF208>


In [21]:
# 会出错
y.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [22]:
# 需要传入相同维数的向量
y.backward(torch.tensor([1, 0.1, 0.01]))

In [23]:
print(x , x.data, x.grad, x.grad_fn)
print(y, y.data, y.grad, y.grad_fn)

tensor([-0.0804, -2.4482, -0.1980], requires_grad=True) tensor([-0.0804, -2.4482, -0.1980]) tensor([-0.1608, -0.4896, -0.0040]) None
tensor([0.0065, 5.9937, 0.0392], grad_fn=<PowBackward0>) tensor([0.0065, 5.9937, 0.0392]) None <PowBackward0 object at 0x0000024A15B07488>


[ -0.8630, -0.3537, -0.0169]各是原来梯度的1、 0.1、 0.01倍

In [24]:
a = torch.randn(3)
a

tensor([0.2410, 0.8845, 0.9085])

In [25]:
a = torch.autograd.Variable(a, requires_grad=True)
a

tensor([0.2410, 0.8845, 0.9085], requires_grad=True)

In [26]:
z = a ** 2

In [28]:
z.backward(torch.tensor([1.0, 1.0, 1.0]))

In [29]:
print(a , a.data, a.grad, a.grad_fn)
print(z, z.data, z.grad, z.grad_fn)

tensor([0.2410, 0.8845, 0.9085], requires_grad=True) tensor([0.2410, 0.8845, 0.9085]) tensor([0.4819, 1.7689, 1.8169]) None
tensor([0.0581, 0.7823, 0.8253], grad_fn=<PowBackward0>) tensor([0.0581, 0.7823, 0.8253]) None <PowBackward0 object at 0x0000024A15B7A988>


## Dataset数据集——torch.utils.data.Dataset

In [36]:
import pandas as pd
class myDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, root_dir, other_file):
        self.csv_data = pd.read_csv(csv_file)
        with open(txt_file, 'r', encoding='utf-8') as f:
            data_list = f.readlines()
        self.txt_data = data_list
        self.root_dir = root_dir
    
    def __len__(self):
        return len(self.csv_data)
    
    def __getitem__(self, idx):
        data = (self.csv_data[idx], self.txt_data[idx])
        return data

通过上面这种方式很难实现取bacth，shuffle或者多线程去读取数据，pytorch提供了一个很简单的方法，torch.utils.data.DataLoader

In [ ]:
from torch.utils.data.dataloader import default_collate
dataiter = torch.utils.data.DataLoader(myDataset, batch_size=32, shuffle=True, collate_fn=default_collate)

## torch.nn.Module（模组）
在pytorch里面编写神经网络，所有的层结构和损失函数都来自于torch.nn，所有的模型构建都是从这个基类nn.Module继承的

In [ ]:
# 模板
import torch.nn as nn
class net_name(nn.Module):
    def __init__(self, other_arguments):
        super(net_name, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size)
        # other layers
    
    def forward(self, x):
        x = self.conv1(x)
        # other layers
        return x

## torch.optim（优化）

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# 优化前需要将梯度归零
optimizer.zeros()
# 然后再反向传播，算出梯度
loss.backward()
# 最后更新参数
optimizer.step()

## 模型的保存
- 保存整个模型的结构和参数,保存的对象是模型model
- 保存模型的参数，保存的对象是模型的状态model.state_dict()


In [ ]:
torch.save(model, 'model.pth')
torch.save(model.state_dict(), 'model_state.pth')

## 模型的加载
- 加载整个模型的结构和参数
- 加载模型的参数，首先需要重构模型的结构

In [ ]:
load_model = torch.load('model.pth')
model.load_state_dict(torch.load('model_state.pth'))